### Approach  
1. Get some random texts from eviction notices (10-30)
2. Augument these random texts to 10000 using text augmentation techniques
3. Randomly generate fake addresses and insert them into specific and random positions in the 10,000 texts
4. At the same time assign the generated fake addresses as a label to that text
5. LSTM
6. User Address parser to extract and label address entities

Submission
- Report
- Code Documentation
- Functions input and Output
- Usage
- Limiyations
- size of input text, output text - pages, words, rows

New IS  
- Finshed trainng dat acreation  
- Modelling part for next quarter  

In [1]:
import pandas as pd
import numpy as np
import usaddress
import pytesseract
import cv2
import glob
import faker
import nltk
import os
import random
try:
    from PIL import Image
except ImportError:
    import Image
import spacy
from py_thesaurus import Thesaurus
from nltk.corpus import wordnet 
import en_core_web_sm
import re
from nlp_aug import *
from nltk.corpus import stopwords
#nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ajink\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Libraries
#### Parse Addresses  
For International addresses: 'https://github.com/openvenues/libpostal'  
For USA Addresses: https://github.com/datamade/usaddress
#### OCR
https://pypi.org/project/pytesseract/
#### Supporting Libraries
cv2, image/PIL.image


### Text Augmentation Techniques  
- Synonym Replacement (SR): Randomly replace n words in the sentences with their synonyms.
- Random Insertion (RI): Insert random synonyms of words in a sentence, this is done n times.
- Random Swap (RS): Two words in the sentences are randomly swapped, this is repeated n-times.
- Random Deletion (RD): Random removal for each word in the sentence with a probability p.  
The formula used to determine the number of sentences augmented is:  
N = Alpha * Length of the sentence.  
Alpha is the “augmentation parameter”, higher the alpha-more aggressive the “EDA”.  (Easy Data Augmentation)
  
Functions: https://www.kaggle.com/init927/nlp-data-augmentation
Paper: https://arxiv.org/pdf/1901.11196.pdf

### Spell Checker
https://rustyonrampage.github.io/text-mining/2017/11/28/spelling-correction-with-python-and-nltk.html

### Approach
1. Creation of Synthetic Data
        1. Gather Text
        2. Convert Text into Documents
        3. Add placeholders to the Text in decided and random locations
            - %%ADDRESS%%, %%NAME%%, %%DATE%%, %%EMAIL%%, %%PHONE%%
        4. Augment the Text along without touching the placeholders
        5. Utility to replace the placeholders with corresponding randomly generated texts
        6. Create a labeled dataset of (Documents, Addresses)


In [2]:
fake=faker.Faker()

In [3]:
img_file_list=glob.glob(r"synthetic-data\*")
print(img_file_list)
img = cv2.imread(r'synthetic-data\image (3).jpg')
print(pytesseract.image_to_string(img))
# OR explicit beforehand converting
#print(pytesseract.image_to_string(Image.fromarray(img))

['synthetic-data\\1575100985_kcaV3Ce9GL_eviction_notice.docx', 'synthetic-data\\1575101025_nJ8RX5bZVI_eviction_notice.docx', 'synthetic-data\\1575101089_qxkhJFBaBW_eviction_notice.docx', 'synthetic-data\\1575101124_Zjdn8Ghl04_eviction_notice.docx', 'synthetic-data\\extract.txt', 'synthetic-data\\image (1).jpg', 'synthetic-data\\image (2).jpg', 'synthetic-data\\image (3).jpg', 'synthetic-data\\sources.txt', 'synthetic-data\\text']
[Apariment Community Name]
[street Address}
[city, State, Zip]
[Phone number and email address]
TenantName(s
Tenant Street Address]
Tenant Apartment Number}
[City, State, Zip]
Dear Mr.Ms.[Tenant LastName(s],
Multiple warnings have been issuedregarding [state reason forissuing eviction
Notice), Yourrental agreement clearly states in [state section of lease containing
violated policy and exactly what was violated), Dueto thetallure onyourpartto
Uupholdthe agreement of your rental the [Apartment Community Name] has no
choice butto submitthis natice of eviction
‘Y

## Data Curation

In [13]:
def read_extract(file_path=r".\synthetic-data\extract.txt"):
    extract=open(file_path,'r',encoding='utf-8')
    extracted=extract.read()
    extract.close()
    return extracted

def fix_extract(extract, threshold=4):
    sentences=nltk.sent_tokenize(extract)
    filtered_sentences=[]
    for sentence in sentences:
        if len(nltk.word_tokenize(sentence))>4:
            filtered_sentences.append(sentence)
    return " ".join(filtered_sentences)
        
def chunk_text(large_chunk,num=10):
    sentences=nltk.sent_tokenize(extracted)
    size=len(sentences)
    mini_chunks=[]
    start=0
    end=num
    for i in range(size//num):
        mini_chunks.append(" ".join(sentences[start:end]))
        start=start+num
        end=end+num
    return mini_chunks

def read_samples(file_path=r".\synthetic-data\text\*"):
    file_list = glob.glob(file_path)
    corpus = []
    for file_path in file_list:
        with open(file_path) as f_input:
            corpus.append(f_input.read())
    return corpus

def insert_placeholder(string, index, placeholder):
    return string[:index] + '\n'+placeholder+'\n' + string[index:]

def randomly_assign_placeholder(mini_chunk,probabilities=[0.8,0.4,0.7], placeholder=r'%%ADDRESS%%'):
    chunk_size=len(mini_chunk)
    if np.random.uniform(0,1)<probabilities[0]:
        index=random.randint(0,chunk_size//3)
        index=index+mini_chunk[index:].find(' ')
        mini_chunk=insert_placeholder(mini_chunk,index,placeholder)
    chunk_size=len(mini_chunk)
    if np.random.uniform(0,1)<probabilities[1]:
        index=random.randint(chunk_size//3,2*chunk_size//3)
        index=index+mini_chunk[index:].find(' ')
        mini_chunk=insert_placeholder(mini_chunk,index,placeholder)
    chunk_size=len(mini_chunk)
    if np.random.uniform(0,1)<probabilities[0]:
        index=random.randint(2*chunk_size//3,chunk_size)
        index=index+mini_chunk[index:].rfind(' ')
        mini_chunk=insert_placeholder(mini_chunk,index,placeholder)
    return mini_chunk

def randomly_assign_placeholder_to_list(mini_chunks,probabilities=[0.8,0.4,0.7],placeholder=r'%%ADDRESS%%'):
    processed_chunks=[]
    for chunk in mini_chunks:
        new_chunk=randomly_assign_placeholder(chunk,probabilities,placeholder)
        processed_chunks.append(new_chunk)
    return processed_chunks

In [5]:
extracted=fix_extract(read_extract())

In [6]:
mini_chunks=chunk_text(extracted,10)

In [7]:
mini_chunks[0]

'An Eviction Notice, also known as a Notice to Quit, is a document sent by a Landlord to a Tenant to inform them of a violation or termination of the lease agreement and to start the process of removing a Tenant from the property. This process can be very technical and varies from state to state. However, this eviction process generally begins with the Landlord providing the Tenant with a written Eviction Notice. Acceptable reasons to evict a tenant range from issues such as nonpayment of rent, violation of the lease terms, engaging in illegal activities on the premises, ending a month to month lease, or a Tenant staying on the premises after the lease has been terminated. The Eviction Notice serves to make the Tenant aware that they have not complied with the terms of the lease or are otherwise subject to being evicted and gives the Tenant a deadline by which they must either correct the issue or make arrangements to leave the property. If the Tenant does not cure the violation or lea

### TEXT AUGMENTATION

1. Synonym Replacement

In [8]:
def synalter_Noun_Verb(word,al,POS):
    max_temp = -1
    flag = 0
    for i in a1:
        try:
            w1 = wordnet.synset(word+'.'+POS+'.01') 
            w2 = wordnet.synset(i+'.'+POS+'.01') # n denotes noun 
            if(max_temp<w1.wup_similarity(w2)):
                max_temp=w1.wup_similarity(w2)
                temp_name = i
                flag =1
        except:
            f = 0
            
    if flag == 0:
        max1 = -1.
        nlp = en_core_web_sm.load()
        for i in a1:
            j=i.replace(' ', '')
            tokens = nlp(u''+j)
            token_main = nlp(u''+word_str)
            for token1 in token_main:
                if max1<float(token1.similarity(tokens)):
                    max1 = token1.similarity(tokens)
                    value = i
        max1 = -1.
        return value 
    else:
        return temp_name


In [9]:
def replace_synonyms(chunks,percent=50):
    output_chunks=[]
    for chunk in chunks:
        output_chunk = chunk
        words = chunk.split()
        counts = {}
        for word in words:
            if word not in counts:
                counts[word] = 0
            counts[word] += 1
        one_word = []
        for key, value in counts.items():
            if value == 1 and key.isalpha() and len(key)>2:
                one_word.append(key)
        noun = []
        verb = []
        nlp = en_core_web_sm.load()
        doc = nlp(u''+' '.join(one_word))
        for token in doc:
            if  token.pos_ == 'VERB':
                verb.append(token.text)
            if  token.pos_ == 'NOUN':
                noun.append(token.text)
        all_main =verb + noun
        len_all = len(noun)+len(verb)
        final_value = int(len_all * percent /100)
        #random.seed(4)
        temp = random.sample(range(0, len_all), final_value)
        for i in temp:
            try:
                word_str = all_main[i]
                w = Word(word_str)
                a1= list(w.synonyms())
                if i<len(verb):
                    change_word=synalter_Noun_Verb(word_str,a1,'v')
                    try:
                        search_word = re.search(r'\b('+word_str+r')\b', output_chunk)
                        Loc = search_word.start()
                        output_chunk = output_chunk[:int(Loc)] + change_word + output_chunk[int(Loc) + len(word_str):]
                    except:
                        f=0

                else:
                    change_word=synalter_Noun_Verb(word_str,a1,'n')
                    try:
                        search_word = re.search(r'\b('+word_str+r')\b', output_chunk)
                        Loc = search_word.start()
                        output_chunk = output_chunk[:int(Loc)] + change_word + output_chunk[int(Loc) + len(word_str):]
                    except:
                        f=0
            except:
                f=0
        output_chunks.append(output_chunk)
    return output_chunks

### Text Augment Functions built upon nlp_aug

In [10]:
def augment_chunk(mini_chunk, probability=0.5):
    sentences=nltk.sent_tokenize(mini_chunk)
    new_sentences=[]
    for sentence in sentences:      
        if random.uniform(0,1)<=probability:
            new_sentence=eda_4(sentences[0], num_aug=1)[0]+'.'
            new_sentences.append(new_sentence)
        else:
            new_sentences.append(sentence)
    return " ".join(new_sentences)

def augment_list_of_chunks(mini_chunks,probability=0.5, num_aug=100):
    augmented_chunks=[]
    num=0
    for i in range(num_aug):
        for chunk in mini_chunks:
            try:
                num=num+1
                augmented_chunks.append(augment_chunk(chunk,probability)) 
            except:
                print("Error in Chunk List: Chunk#" +str(num-1))
                return None
    augmented_chunks.extend(mini_chunks)
    return augmented_chunks

### Augment chunks derived from the text extract

In [11]:
ag_chunks=augment_list_of_chunks(mini_chunks, num_aug=2)

### Assignment Address placeholders to the augmented chunks derived from the text extract

In [14]:
processed_chunks=randomly_assign_placeholder_to_list(ag_chunks)

### Get Samples and Augment Them

In [15]:
sample_notices=read_samples()
aug_sample_notices=augment_list_of_chunks(sample_notices, num_aug=50)

### Combine the two datasets

In [16]:
synthetic_data=processed_chunks+aug_sample_notices

### Replace %%ADDRESS%%  placeholders with random fake addresses

In [17]:
def count_occurences(source_string, substring):
    return re.subn(substring, '', source_string)[1]   

https://stackoverflow.com/questions/27589325/how-to-find-and-replace-nth-occurrence-of-word-in-a-sentence-using-python-regula

In [18]:
def replace_ith_instance(string, pattern, new_str, i = None, pattern_flags = 0):
    # If i is None - replacing last occurrence
    match_obj = re.finditer(r'{0}'.format(pattern), string, flags = pattern_flags)
    matches = [item for item in match_obj]
    if i == None:
        i = len(matches)
    if len(matches) == 0 or len(matches) < i:
        return string
    match = matches[i - 1]
    match_start_index = match.start()
    match_len = len(match.group())
    return '{0}{1}{2}'.format(string[0:match_start_index], new_str, string[match_start_index + match_len:])

In [19]:
def replace_placeholders(chunk,placeholder='%%ADDRESS%%'):
    fake=faker.Faker()
    num_addr=count_occurences(chunk,placeholder)
    addresses=[]
    for i in range(num_addr):
        fake_addr=fake.address()
        chunk=replace_ith_instance(chunk,placeholder,fake_addr,1)
        addresses.append(fake_addr)
    return (chunk, tuple(addresses))

def get_labeled_data(chunk_list):
    labeled_data=[]
    for chunk in chunk_list:
        labeled_data.append(replace_placeholders(chunk))
    return labeled_data

In [30]:
c_test=synthetic_data[0]

In [22]:
labeled_data=get_labeled_data(synthetic_data)

In [ ]:
#test
print(labeled_data[0])
print(c_test)

### Modelling

https://towardsdatascience.com/addressnet-how-to-build-a-robust-street-address-parser-using-a-recurrent-neural-network-518d97b9aebd  
https://www.tensorflow.org/tutorials/text/text_classification_rnn  
https://keras.io/examples/lstm_seq2seq/  

('87268 Gabrielle Roads\nLake John, MT 60899',
 '13060 Sheila Stream Apt. 293\nEast Dennis, NJ 03143',
 '184 Donald Rapids Apt. 621\nSouth Kyle, AZ 62171')

In [42]:
labeled_data_df=pd.DataFrame(labeled_data,columns=['Notice','Addresses'])

In [45]:
labeled_data_df.to_csv('.\labeled_data.txt',sep='|',index=False)